In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from itertools import product

In [2]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        t = F.max_pool2d(F.relu(self.conv1(t)), kernel_size=2, stride=2)  # Layer 1
        t = F.max_pool2d(F.relu(self.conv2(t)), kernel_size=2, stride=2)  # Layer 2
        t = t.reshape(-1, 12*4*4)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root = 'data',
    download=False,
    train = True,
    transform=transforms.Compose([transforms.ToTensor()])
)

In [6]:
# tb = SummaryWriter()

# network = Network()
# images, labels = next(iter(train_loader))
# grid = torchvision.utils.make_grid(images)

# tb.add_image('images', grid)
# tb.add_graph(network, images)
# tb.close()

In [7]:
parameters = dict(
batch_size = [100,1000],
lr = [0.01,0.001],
shuffle = [True, False]
)
param_values = [v for v in parameters.values()]

for batch_size, lr, shuffle in product(*param_values):
    network = Network().to('cuda')
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=shuffle)
    optimizer = optim.Adam(network.parameters(), lr=lr)

    comment = f' batch_size={batch_size} lr={lr} shuffle={shuffle}'
    tb = SummaryWriter(comment=comment)

    for epoch in range(2):
        total_loss=0
        total_correct=0

        for batch in train_loader:
            images, labels = batch.to('cuda')
            preds = network(images)

            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            total_loss+=loss.item() * batch_size
            total_correct+=get_num_correct(preds, labels)

        tb.add_scalar('Loss', total_loss, epoch)
        tb.add_scalar('Number Correct', total_correct, epoch)
        tb.add_scalar('Accuracy', total_correct/len(train_set), epoch)

        print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)

    tb.close()

AttributeError: 'list' object has no attribute 'to'